In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import re
from datetime import date, timedelta
import time
import mysql.connector
from pandas.io import sql
import sqlalchemy


In [20]:
flag = 0
end_date = ''

database_username = ''
database_password = ''
database_ip       = ''
database_name     = ''
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

def get_weather(sdate, edate):

    names = ['Date', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score', 'Venue', 'Weather (Degrees)', 'Weather Type', 'Wind (MPH)', 'Wind Direction']
    df = pd.DataFrame(columns = names)  

    delta = edate - sdate

    failures = []

    count = 0

    driver = webdriver.Chrome(executable_path = 'C:\Drivers\Chrome_driver\chromedriver')

    for i in range(delta.days + 1):
        links = []
        day = sdate + timedelta(days=i)
        url = 'https://www.mlb.com/scores/' + str(day)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser') 
        boxes = soup.find_all('a', class_ = re.compile('trk-box'))
        for box in boxes:
            if str(box.parent.parent.parent).find('czAVYt') == -1:
                links.append(box['href'])
        for link in links:
            driver.get(link)
            time.sleep(3)
            driver.maximize_window()
            data = driver.page_source
            soup = BeautifulSoup(data, 'html.parser')
            try:
                header = soup.find('div', class_ = 'header')
                header = header.text
                header = re.split(', | ', header)
                if not header[1].isdigit():
                    header[0 : 2] = [' '.join(header[0 : 2])] 
                if not header[-3].isdigit():
                    header[-3 : -1] = [' '.join(header[-3 : -1])]
                venue = soup.find('span', text = 'Venue').parent
                venue = venue.text
                venue = venue.replace('Venue: ', '')
                venue = venue.replace('.', '')
                weather = soup.find('span', text = 'Weather').parent
                weather = weather.text
                weather = weather.replace('Weather: ', '')
                weather = weather.replace(' degrees', '')
                weather = weather.replace('.', '')
                weather = weather.split(', ')
                wind = soup.find('span', text = 'Wind').parent
                wind = wind.text
                wind = wind.replace('Wind: ', '')
                wind = wind.replace(' mph', '')
                wind = wind.replace('.', '')
                wind = wind.split(', ')
                df.at[count, 'Date'] = day
                df.at[count, 'Team 1'] = header[2]
                df.at[count, 'Team 2'] = header[0]
                df.at[count, 'Team 1 Score'] = header[3] 
                df.at[count, 'Team 2 Score'] = header[1]
                df.at[count, 'Venue'] = venue
                df.at[count, 'Weather (Degrees)'] = weather[0]
                df.at[count, 'Weather Type'] = weather[1]
                df.at[count, 'Wind (MPH)'] = wind[0]
                df.at[count, 'Wind Direction'] = wind[1]
                count += 1
            except:
                failures.append(link)
    driver.close()
    return df
    
connection = database_connection.connect()
while flag == 0:
    weatherA = get_weather(date.today(), date.today())
    try:
        weatherA.to_sql('Weather', connection, index= False, if_exists= 'append')
        print("Success")
    except:
        print('Error on ' + str(date.today()))
    finally:
        if date.today() == end_date:
            flag = 1
        if flag == 0:
            time.sleep(86400)
connection.close()

Success
